In [5]:
# Importar bibliotecas y cargar datos

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar la base de datos
df = pd.read_csv('dataset/modelo.csv')

# Crear un vectorizador TF-IDF para convertir títulos en vectores numéricos
# Crear una matriz TF-IDF para representar los juegos
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['title'] + ' ' + df['tags'] + ' ' + df['genres'])

# Calcular la similitud de coseno entre los juegos
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Función para obtener juegos recomendados (limitados a 5)
def recomendacion_juego(title, num_recommendations=5):
    # Comprobar si 'title' es una cadena
    if not isinstance(title, str):
        return "El título debe ser una cadena."

    # Comprobar si 'title' existe en la lista de títulos
    if title not in df['title'].values:
        return "El título proporcionado no se encuentra en la lista de títulos."

    idx = df[df['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]  # Obtener las 'num_recommendations' mejores recomendaciones
    game_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[game_indices].tolist()  # Convertir los resultados a una lista y retornarla


def recomendacion_usuario(user_id, num_recommendations=5):
    # Verificar si 'user_id' es una cadena
    if not isinstance(user_id, str):
        return "El ID de usuario debe ser una cadena."

    # Filtrar el DataFrame para obtener los juegos que le gustan al usuario
    user_likes = df[df['user_id'] == user_id]
    
    if user_likes.empty:
        return "No se encontraron juegos que le gusten al usuario."

    # Calcular el promedio de las puntuaciones de similitud para los juegos que le gustan al usuario
    avg_similarity_scores = cosine_sim[user_likes.index].mean(axis=0)

    # Obtener los índices de los juegos recomendados ordenados por similitud
    game_indices = avg_similarity_scores.argsort()[::-1]

    # Excluir los juegos que ya le gustan al usuario
    recommended_games = [game for game in game_indices if game not in user_likes.index]

    # Tomar los primeros 'num_recommendations' juegos recomendados
    recommended_games = recommended_games[:num_recommendations]

    # Obtener los títulos de los juegos recomendados
    recommended_titles = df.iloc[recommended_games]['title'].tolist()

    return recommended_titles

In [6]:
related_games = recomendacion_juego('Call of Duty: World at War')
print(related_games)

['Call of Duty® 2', 'Call of Duty®', 'Call of Duty: United Offensive', 'Call of Duty®: Black Ops', 'Call of Duty®: Black Ops II']


In [7]:
# Ejemplo de uso
recommended_games = recomendacion_usuario('Bennysaputra')
print(recommended_games)

['Counter-Strike: Source', 'Counter-Strike: Condition Zero', 'Counter-Strike: Global Offensive', 'Insurgency', 'Team Fortress Classic']
